In [1]:
! pip install requests


In [ ]:
# Grab real flight info from aviationstack by using their API
import requests

API_KEY = "dbc03ac16b1ce1524a4844dba4e7cb88"  
BASE_URL = "http://api.aviationstack.com/v1/flights"

params = {
    'access_key': API_KEY,
    'dep_iata': 'ORD',       
    'arr_iata': 'JFK',      
    'limit': 10              
}

response = requests.get(BASE_URL, params=params)
data = response.json()

# print flight
for flight in data['data']:
    print({
        "flight_number": flight['flight']['iata'],
        "airline": flight['airline']['name'],
        "departure_city": flight['departure']['airport'],
        "departure_time": flight['departure']['scheduled'],
        "arrival_city": flight['arrival']['airport'],
        "arrival_time": flight['arrival']['scheduled'],
        "departure_terminal": flight['departure']['terminal'],
        "arrival_terminal": flight['arrival']['terminal'],
        "departure_airport": flight['departure']['iata'],
        "arrival_airport": flight['arrival']['iata'],
        "flight_status": flight['flight_status']
    })


{'flight_number': 'AA532', 'airline': 'American Airlines', 'departure_city': "Chicago O'hare International", 'departure_time': '2025-06-19T10:03:00+00:00', 'arrival_city': 'John F Kennedy International', 'arrival_time': '2025-06-19T13:28:00+00:00', 'departure_terminal': '3', 'arrival_terminal': '8', 'departure_airport': 'ORD', 'arrival_airport': 'JFK', 'flight_status': 'landed'}
{'flight_number': 'DL5227', 'airline': 'Delta Air Lines', 'departure_city': "Chicago O'hare International", 'departure_time': '2025-06-19T10:35:00+00:00', 'arrival_city': 'John F Kennedy International', 'arrival_time': '2025-06-19T13:59:00+00:00', 'departure_terminal': '5', 'arrival_terminal': '4', 'departure_airport': 'ORD', 'arrival_airport': 'JFK', 'flight_status': 'landed'}
{'flight_number': 'B6906', 'airline': 'JetBlue Airways', 'departure_city': "Chicago O'hare International", 'departure_time': '2025-06-19T11:13:00+00:00', 'arrival_city': 'John F Kennedy International', 'arrival_time': '2025-06-19T14:35:0

In [4]:
! pip install amadeus


  Preparing metadata (setup.py) ... done
  Created wheel for amadeus: filename=amadeus-12.0.0-py2.py3-none-any.whl size=67477 sha256=fb7314f3dd064ef5c49011cae564346fa5deccb36e1427977d95ca8d5e5b7f5b
  Stored in directory: /Users/liuyueyang/Library/Caches/pip/wheels/12/3d/ca/0c01ed3dde8eadb37517c80c1bab31308fc99f0ee31e539850
Successfully built amadeus


In [ ]:
from amadeus import Client, ResponseError

amadeus = Client(
    client_id='D1s8Uk3wKoDsHoghWtYbGrhqGM8tj3r4',  
    client_secret='qlOZ6Xik94UfNEmI',                  
    hostname='test'                             
)

try:
    response = amadeus.shopping.flight_offers_search.get(
        originLocationCode='ORD',       
        destinationLocationCode='JFK',
        departureDate='2025-06-25',
        adults=1,
        max=3
    )

    for offer in response.data:
        itinerary = offer['itineraries'][0]
        segments = itinerary['segments']
        first = segments[0]
        last = segments[-1]
        
        print({
            "price_usd": offer['price']['total'],
            "direct": len(segments) == 1,
            "departure_airport": first['departure']['iataCode'],
            "arrival_airport": last['arrival']['iataCode'],
            "departure_time": first['departure']['at'],
            "arrival_time": last['arrival']['at'],
            "duration": itinerary['duration'],
            "airline": first['carrierCode'],
            "flight_number": first['number'],
            "layover_city": segments[1]['departure']['iataCode'] if len(segments) > 1 else None
        })

except ResponseError as error:
    print(error)


{'price_usd': '154.43', 'direct': False, 'departure_airport': 'ORD', 'arrival_airport': 'JFK', 'departure_time': '2025-06-25T22:11:00', 'arrival_time': '2025-06-26T11:59:00', 'duration': 'PT12H48M', 'airline': 'F9', 'flight_number': '3243', 'layover_city': 'DFW'}
{'price_usd': '154.43', 'direct': False, 'departure_airport': 'ORD', 'arrival_airport': 'JFK', 'departure_time': '2025-06-25T16:21:00', 'arrival_time': '2025-06-26T10:00:00', 'duration': 'PT16H39M', 'airline': 'F9', 'flight_number': '3044', 'layover_city': 'MCO'}
{'price_usd': '154.43', 'direct': False, 'departure_airport': 'ORD', 'arrival_airport': 'JFK', 'departure_time': '2025-06-25T09:01:00', 'arrival_time': '2025-06-26T09:31:00', 'duration': 'PT23H30M', 'airline': 'F9', 'flight_number': '1416', 'layover_city': 'TPA'}


In [ ]:
from amadeus import Client, ResponseError
import random
import pandas as pd

amadeus = Client(
    client_id='D1s8Uk3wKoDsHoghWtYbGrhqGM8tj3r4',
    client_secret='qlOZ6Xik94UfNEmI',
    hostname='test'
)

routes = [
    ("ORD", "JFK"),
    ("ORD", "LAX"),
    ("ORD", "SFO"),
    ("ORD", "MIA"),
    ("ORD", "BOS"),
    ("ORD", "SEA"),
    ("ORD", "IAD")
]

flights = []

for origin, destination in routes:
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=origin,
            destinationLocationCode=destination,
            departureDate='2025-06-25',
            adults=1,
            max=10
        )

        for offer in response.data:
            itinerary = offer['itineraries'][0]
            segments = itinerary['segments']
            first = segments[0]
            last = segments[-1]

            # Convert duration like 'PT2H30M' to minutes
            duration_str = itinerary['duration']
            if "H" in duration_str:
                hours = int(duration_str.split("H")[0].replace("PT", ""))
                minutes = int(duration_str.split("H")[1].replace("M", ""))
                duration_minutes = hours * 60 + minutes
            else:
                duration_minutes = int(duration_str.replace("PT", "").replace("M", ""))

            flights.append({
                "price_usd": float(offer['price']['total']),
                "direct": len(segments) == 1,
                "departure_airport": first['departure']['iataCode'],
                "arrival_airport": last['arrival']['iataCode'],
                "arrival_terminal": None,
                "departure_time": first['departure']['at'],
                "arrival_time": last['arrival']['at'],
                "duration_minutes": duration_minutes,
                "flight_number": f"{first['carrierCode']}{first['number']}",
                "layover_city": None if len(segments) == 1 else segments[1]['departure']['iataCode'],
                "departure_city": origin,
                "arrival_city": destination,
                "wifi_available": random.choice([True, False]),
                "source": "real"
            })

    except ResponseError as e:
        print(f"{origin} → {destination} 出错: {e}")

df = pd.DataFrame(flights)
df.to_csv("/Users/liuyueyang/Desktop/real_amadeus_flights.csv", index=False)



保存完成！共拉取航班数： 67


In [69]:
data=pd.read_csv("/Users/liuyueyang/Desktop/United_Airlines_-_10K_Realistic_Synthetic_Flights.csv")
data.head(10)

,price_usd,direct,departure_airport,arrival_airport,arrival_terminal,departure_time,arrival_time,duration_minutes,flight_number,layover_city,departure_city,arrival_city,wifi_available,source
0,154.43,False,ORD,JFK,NaN,2025-06-25T22:11:00,2025-06-26T11:59:00,768,F93243,DFW,ORD,JFK,True,real
1,154.43,False,ORD,JFK,NaN,2025-06-25T16:21:00,2025-06-26T10:00:00,999,F93044,MCO,ORD,JFK,False,real
2,154.43,False,ORD,JFK,NaN,2025-06-25T09:01:00,2025-06-26T09:31:00,1410,F91416,TPA,ORD,JFK,True,real
3,216.43,True,ORD,JFK,NaN,2025-06-25T06:00:00,2025-06-25T09:12:00,132,B6406,NaN,ORD,JFK,True,real
4,532.48,True,ORD,JFK,NaN,2025-06-25T11:35:00,2025-06-25T14:58:00,143,B6906,NaN,ORD,JFK,True,real
5,588.73,False,ORD,JFK,NaN,2025-06-25T17:51:00,2025-06-26T08:34:00,823,B61012,BOS,ORD,JFK,False,real
6,959.60,False,ORD,JFK,NaN,2025-06-25T17:51:00,2025-06-26T06:40:00,709,B61012,BOS,ORD,JFK,False,real
7,160.33,False,ORD,LAX,NaN,2025-06-25T05:35:00,2025-06-25T09:56:00,381,F91483,DEN,ORD,LAX,True,real
8,165.18,False,ORD,LAX,NaN,2025-06-25T22:11:00,2025-06-26T07:33:00,682,F93243,DFW,ORD,LAX,True,real
9,165.18,False,ORD,LAX,NaN,2025-06-25T05:35:00,2025-06-26T00:27:00,1252,F91483,DEN,ORD,LAX,False,real


In [70]:
# Update all flight numbers to be United Airlines (UA) format
data["flight_number"] = ["UA" + str(random.randint(100, 9999)).zfill(4) for _ in range(len(data))]

# Confirm all are UA flights
data["flight_number"].head()


0    UA8844
1    UA2027
2    UA1989
3    UA4414
4    UA9009
Name: flight_number, dtype: object

In [71]:
data[data["source"]=="real"].head(10)

,price_usd,direct,departure_airport,arrival_airport,arrival_terminal,departure_time,arrival_time,duration_minutes,flight_number,layover_city,departure_city,arrival_city,wifi_available,source
0,154.43,False,ORD,JFK,NaN,2025-06-25T22:11:00,2025-06-26T11:59:00,768,UA8844,DFW,ORD,JFK,True,real
1,154.43,False,ORD,JFK,NaN,2025-06-25T16:21:00,2025-06-26T10:00:00,999,UA2027,MCO,ORD,JFK,False,real
2,154.43,False,ORD,JFK,NaN,2025-06-25T09:01:00,2025-06-26T09:31:00,1410,UA1989,TPA,ORD,JFK,True,real
3,216.43,True,ORD,JFK,NaN,2025-06-25T06:00:00,2025-06-25T09:12:00,132,UA4414,NaN,ORD,JFK,True,real
4,532.48,True,ORD,JFK,NaN,2025-06-25T11:35:00,2025-06-25T14:58:00,143,UA9009,NaN,ORD,JFK,True,real
5,588.73,False,ORD,JFK,NaN,2025-06-25T17:51:00,2025-06-26T08:34:00,823,UA2943,BOS,ORD,JFK,False,real
6,959.60,False,ORD,JFK,NaN,2025-06-25T17:51:00,2025-06-26T06:40:00,709,UA3925,BOS,ORD,JFK,False,real
7,160.33,False,ORD,LAX,NaN,2025-06-25T05:35:00,2025-06-25T09:56:00,381,UA0765,DEN,ORD,LAX,True,real
8,165.18,False,ORD,LAX,NaN,2025-06-25T22:11:00,2025-06-26T07:33:00,682,UA5453,DFW,ORD,LAX,True,real
9,165.18,False,ORD,LAX,NaN,2025-06-25T05:35:00,2025-06-26T00:27:00,1252,UA8161,DEN,ORD,LAX,False,real


In [72]:

# Create a mapping from IATA airport code to city name for correction
iata_to_city = {
    "ORD": "Chicago", "JFK": "New York", "IAD": "Washington", "LAX": "Los Angeles", "SFO": "San Francisco",
    "MIA": "Miami", "BOS": "Boston", "DFW": "Dallas", "ATL": "Atlanta", "PHX": "Phoenix", "LAS": "Las Vegas",
    "MCO": "Orlando", "SEA": "Seattle", "DEN": "Denver", "PHL": "Philadelphia", "TPA": "Tampa",
    "IND": "Indianapolis", "PIT": "Pittsburgh"
}

# Apply mapping to correct any airport codes shown as cities
data["departure_city"] = data["departure_city"].map(iata_to_city).fillna(data["departure_city"])
data["arrival_city"] = data["arrival_city"].map(iata_to_city).fillna(data["arrival_city"])
data["layover_city"] = data["layover_city"].map(iata_to_city).fillna(data["layover_city"])

data.head(10)


,price_usd,direct,departure_airport,arrival_airport,arrival_terminal,departure_time,arrival_time,duration_minutes,flight_number,layover_city,departure_city,arrival_city,wifi_available,source
0,154.43,False,ORD,JFK,NaN,2025-06-25T22:11:00,2025-06-26T11:59:00,768,UA8844,Dallas,Chicago,New York,True,real
1,154.43,False,ORD,JFK,NaN,2025-06-25T16:21:00,2025-06-26T10:00:00,999,UA2027,Orlando,Chicago,New York,False,real
2,154.43,False,ORD,JFK,NaN,2025-06-25T09:01:00,2025-06-26T09:31:00,1410,UA1989,Tampa,Chicago,New York,True,real
3,216.43,True,ORD,JFK,NaN,2025-06-25T06:00:00,2025-06-25T09:12:00,132,UA4414,NaN,Chicago,New York,True,real
4,532.48,True,ORD,JFK,NaN,2025-06-25T11:35:00,2025-06-25T14:58:00,143,UA9009,NaN,Chicago,New York,True,real
5,588.73,False,ORD,JFK,NaN,2025-06-25T17:51:00,2025-06-26T08:34:00,823,UA2943,Boston,Chicago,New York,False,real
6,959.60,False,ORD,JFK,NaN,2025-06-25T17:51:00,2025-06-26T06:40:00,709,UA3925,Boston,Chicago,New York,False,real
7,160.33,False,ORD,LAX,NaN,2025-06-25T05:35:00,2025-06-25T09:56:00,381,UA0765,Denver,Chicago,Los Angeles,True,real
8,165.18,False,ORD,LAX,NaN,2025-06-25T22:11:00,2025-06-26T07:33:00,682,UA5453,Dallas,Chicago,Los Angeles,True,real
9,165.18,False,ORD,LAX,NaN,2025-06-25T05:35:00,2025-06-26T00:27:00,1252,UA8161,Denver,Chicago,Los Angeles,False,real


In [73]:
data["departure_time"] = pd.to_datetime(data["departure_time"],format='mixed')
data["arrival_time"] = pd.to_datetime(data["arrival_time"],format='mixed')


In [74]:
data.tail(10)

,price_usd,direct,departure_airport,arrival_airport,arrival_terminal,departure_time,arrival_time,duration_minutes,flight_number,layover_city,departure_city,arrival_city,wifi_available,source
9990,680.97,True,IAD,MIA,T3,2025-06-25 20:15:00,2025-06-26 00:38:00,263,UA6713,NaN,Washington,Miami,False,synthetic
9991,471.86,True,SFO,MIA,T3,2025-08-05 06:15:00,2025-08-05 08:48:00,153,UA7194,NaN,San Francisco,Miami,False,synthetic
9992,271.28,False,MIA,ATL,T5,2025-07-01 01:15:00,2025-07-01 05:45:00,270,UA7935,Denver,Miami,Atlanta,True,synthetic
9993,247.08,True,ORD,DEN,T3,2025-08-06 12:15:00,2025-08-06 14:16:00,121,UA2609,NaN,Chicago,Denver,False,synthetic
9994,173.79,False,LAS,MCO,T2,2025-08-13 05:15:00,2025-08-13 09:58:00,283,UA9675,Denver,Las Vegas,Orlando,True,synthetic
9995,490.02,True,BOS,MCO,T3,2025-07-12 04:15:00,2025-07-12 08:33:00,258,UA5717,NaN,Boston,Orlando,True,synthetic
9996,572.83,True,LAS,MCO,T2,2025-07-14 20:15:00,2025-07-15 01:19:00,304,UA4084,NaN,Las Vegas,Orlando,False,synthetic
9997,443.98,True,MCO,DEN,T3,2025-07-11 18:15:00,2025-07-11 23:44:00,329,UA6602,NaN,Orlando,Denver,True,synthetic
9998,482.12,True,ORD,LAX,T1,2025-07-19 03:15:00,2025-07-19 08:01:00,286,UA4812,NaN,Chicago,Los Angeles,False,synthetic
9999,792.45,False,LAX,ORD,T1,2025-07-02 07:15:00,2025-07-02 10:32:00,197,UA6972,Philadelphia,Los Angeles,Chicago,True,synthetic


In [76]:
data['arrival_terminal'] = data['arrival_terminal'].fillna('T1')
missing_counts = data.isna().sum()
print(missing_counts)
    

price_usd               0
direct                  0
departure_airport       0
arrival_airport         0
arrival_terminal        0
departure_time          0
arrival_time            0
duration_minutes        0
flight_number           0
layover_city         7963
departure_city          0
arrival_city            0
wifi_available          0
source                  0
dtype: int64


In [25]:
! pip install ace_tools

In [ ]:
# Separate columns by type

import pandas as pd



numerical_cols = ["price_usd", "duration_minutes"]
categorical_cols = ["direct", "wifi_available", "departure_city", "arrival_city", "layover_city","departure_airport",'arrival_airport','arrival_terminal']
date_cols = ["departure_time", "arrival_time"]

# 1. Numerical distribution (summary stats)
numerical_summary = data[numerical_cols].describe().T

# 2. Date range

date_summary = {
    "departure_time_min": pd.to_datetime(data["departure_time"]).min(),
    "departure_time_max": pd.to_datetime(data["departure_time"]).max(),
    "arrival_time_min": pd.to_datetime(data["arrival_time"]).min(),
    "arrival_time_max": pd.to_datetime(data["arrival_time"]).max()
}

# 3. Categorical frequency (top values)
categorical_frequency = {}
for col in categorical_cols:
    freq_table = data[col].value_counts().reset_index()
    freq_table.columns = [col, "count"]
    categorical_frequency[col] = freq_table

print("📊 Numerical Summary:")
print(numerical_summary)

print("\n🧭 Categorical Frequencies:")
for key, df in categorical_frequency.items():
    print(f"\n🧭 {key} Frequency Table:")
    display(df) 

print("\n📅 Date Ranges:")
for k, v in date_summary.items():
    print(f"{k}: {v}")

📊 Numerical Summary:
                    count        mean         std     min      25%      50%       75%     max
price_usd         10000.0  449.186284  201.720114  100.03  276.895  448.165  623.9075   959.6
duration_minutes  10000.0  228.159200   96.281835   90.00  157.000  225.000  294.0000  1748.0

🧭 Categorical Frequencies:

🧭 direct Frequency Table:


,direct,count
0,True,7963
1,False,2037



🧭 wifi_available Frequency Table:


,wifi_available,count
0,False,5025
1,True,4975



🧭 departure_city Frequency Table:


...


🧭 arrival_airport Frequency Table:


,arrival_airport,count
0,ATL,704
1,PHL,695
2,IAD,692
3,DEN,685
4,SEA,676
5,MIA,674
6,BOS,668
7,JFK,666
8,DFW,657
9,MCO,656



🧭 arrival_terminal Frequency Table:


,arrival_terminal,count
0,T5,2037
1,T1,2020
2,T4,2014
3,T3,1993
4,T2,1936



📅 Date Ranges:
departure_time_min: 2025-06-20 21:15:00
departure_time_max: 2025-08-19 20:15:00
arrival_time_min: 2025-06-20 23:05:00
arrival_time_max: 2025-08-20 02:12:00


In [79]:
df = pd.DataFrame(data)
df.to_csv("/Users/liuyueyang/Desktop/Mock_flights.csv", index=False)
